In [1]:
from my_ppo_2 import PPO
import gym
import importlib

In [2]:
env=gym.make('Pendulum-v0')

In [3]:
state_dim=env.observation_space.shape[0]
action_dim=env.action_space.shape[0]

In [4]:
model=PPO(state_dim,action_dim,8192)

In [5]:
for i in range(1000):
    s=env.reset()
    d=False
    while not d:
        a,action,logprob,statevalue=model.select_action(s)
        s_,r,d,_=env.step(a)
        model.buffer.store(s,action,logprob,s_,r,statevalue,d)
    if i%10==0:
        model.update(200)
        print(i/10,' update')

0.0  update
1.0  update
2.0  update
3.0  update
4.0  update
5.0  update
6.0  update
7.0  update
8.0  update
9.0  update
10.0  update
11.0  update
12.0  update
13.0  update
14.0  update
15.0  update
16.0  update
17.0  update
18.0  update
19.0  update
20.0  update
21.0  update
22.0  update
23.0  update
24.0  update
25.0  update
26.0  update
27.0  update
28.0  update
29.0  update
30.0  update
31.0  update
32.0  update
33.0  update
34.0  update
35.0  update
36.0  update
37.0  update
38.0  update
39.0  update
40.0  update
41.0  update
42.0  update
43.0  update
44.0  update
45.0  update
46.0  update
47.0  update
48.0  update
49.0  update
50.0  update
51.0  update
52.0  update
53.0  update
54.0  update
55.0  update
56.0  update
57.0  update
58.0  update
59.0  update
60.0  update
61.0  update
62.0  update
63.0  update
64.0  update
65.0  update
66.0  update
67.0  update
68.0  update
69.0  update
70.0  update
71.0  update
72.0  update
73.0  update
74.0  update
75.0  update
76.0  update
77.0  upd

In [6]:
i=model.buffer.sample(10,True)


In [13]:
import numpy as np
import torch
import random

In [9]:
a=torch.stack(model.buffer.state)

In [6]:
from reward_env import RewardEnv

In [10]:
senv=RewardEnv(env.observation_space,env.action_space)

In [16]:
model=PPO(senv.observation_space.shape[0],senv.action_space.shape[0],8192)

In [22]:
for i in range(1000):
    s=senv.reset()
    d=False
    while not d:
        a,action,l,v=model.select_action(s)
        s_,r,d,_=senv.step(a)
        model.buffer.store(s,action,l,s_,r,v,d)
        s=s_
        print(d)
    if i%10==0 & i!=0:
        model.update(10)
        print(i/10,' update')

True
True
True
True
False
False
False
True
False
False
False
True
True
False
False
True
True
False
False
True
False
True
True
True
True
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
True
False
False
False
True
False
True
False
False
False
True
False
True
True
True
False
True
True
True
False
False
False
False
True
False
True
False
False
True
True
False
True
False
False
True
True
True
False
False
False
True
False
False
True
True
False
False
False
False
True
True
False
True
False
True
True
True
True
True
False
True
False
True
False
False
True
False
False
False
False
True
False
True
True
True
True
False
False
True
False
True
False
True
False
True
True
False
True
True
False
True
True
False
True
True
True
True
False
True
True
True
True
False
False
False
True
False
True
True
False
True
False
True
False
False
False
False
True
False
True
True
True
True
True
True
True
False
True
False
False
False
False
True
True
True
True
True
True
False
False
True
True


In [20]:
model.buffer.state

[tensor([ 0.3262,  0.8987, -1.7610, -0.1067], device='cuda:0'),
 tensor([ 0.8931, -0.9378, -7.1909,  0.4347], device='cuda:0'),
 tensor([ 0.7189,  0.3394, -0.1166, -1.9049], device='cuda:0'),
 tensor([ 0.1805, -0.4716, -4.7897,  1.2949], device='cuda:0'),
 tensor([-0.5713,  0.8226,  3.2058,  0.2054], device='cuda:0'),
 tensor([-0.2256,  0.8446, -7.7210,  0.7935], device='cuda:0'),
 tensor([-0.4543, -0.1215, -2.8190, -0.5615], device='cuda:0'),
 tensor([ 0.3566, -0.3328, -6.9352,  0.6661], device='cuda:0'),
 tensor([ 0.3676, -0.6152,  5.2833, -0.6648], device='cuda:0'),
 tensor([ 0.1800, -0.9057, -2.1046,  1.0230], device='cuda:0'),
 tensor([-0.9343, -0.2256,  2.3901, -0.2920], device='cuda:0'),
 tensor([-0.0442, -0.5817,  4.8335, -1.8219], device='cuda:0'),
 tensor([-0.4563,  0.5074, -2.6830,  1.8922], device='cuda:0'),
 tensor([ 0.2000,  0.7471, -2.9396, -0.1112], device='cuda:0'),
 tensor([-0.2701, -0.6767,  3.3947,  0.1822], device='cuda:0'),
 tensor([ 0.3625, -0.8409,  0.0592,  1.1